# Ingestion
This notebook parses data from XXX using Google Vertex AI Generative AI.  It then uses Generative AI to create Neo4j Cypher queries which write the data to a Neo4j database.

## Setup
This notebook should be run within Vertex AI Workbench.  Be sure to select "single user" when starting a managed notebook to run this.  Otherwise the auth won't allow access to the preview.  

First we need to install the preview libraries for Generative AI.  It's a new version of the AI platform library.  To access the bucket, your user account and project will need to be part of the preview.

By default a Vertex AI Workbench Notebook uses a service account.  That account doesn't have access to the bucket where the preview binary is.  So, you'll need to auth.  To do so, open a terminal window in your managed notebook and run the command 'gcloud auth login'.  With that complete, you'll be able to install the preview library.

In [3]:
%%capture
!pip install google-cloud-aiplatform --upgrade

Let us now fine-tune the `text-bison` model to help us extract entities & relationships better. The untuned model makes is good but we tune here to make it better.

In [4]:
# Note, you will need to set these variables
project_id = 'neo4jbusinessdev'
location = 'us-central1'

In [5]:
import vertexai
vertexai.init(project=project_id, location=location)

ModuleNotFoundError: No module named 'vertexai'

We will use a `jsonl` file in the following format to tune our Ingestion model. Our ingestion model will consume text and extract entities & relationships out of it. Each training example should be JSONL record with two keys, for example:

```
{
  "input_text": <input prompt>,
  "output_text": <associated output>
}
```

Model tuning will take a few minutes and consumes TPU resources. If the model is already tuned, you can skip the below cell

In [106]:
training_data = 'gs://gs_vertex_ai/entity-extraction-trng/entity-extraction-trng-1.jsonl'
train_steps = 10

vertexai.init(project=project_id, location=location)
model = TextGenerationModel.from_pretrained("text-bison@001")

model.tune_model(
  training_data=training_data,
  train_steps=train_steps,
  tuning_job_location="europe-west4",
  tuned_model_location="us-central1",
)

Creating PipelineJob
PipelineJob created. Resource name: projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524054242
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524054242')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west4/pipelines/runs/tune-large-model-20230524054242?project=803648085855
PipelineJob projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524054242 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524054242 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524054242 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/803648085855/locations/europe-west4/pi

In [107]:
from vertexai.preview.language_models import TextGenerationModel

model = TextGenerationModel.from_pretrained("text-bison@001")
entity_extraction_tuned_model = model.list_tuned_model_names()
print(entity_extraction_tuned_model)

['projects/803648085855/locations/us-central1/models/1971567285113978880', 'projects/803648085855/locations/us-central1/models/3124488789720825856', 'projects/803648085855/locations/us-central1/models/2671877027170091008', 'projects/803648085855/locations/us-central1/models/7947351409425383424']


## Data Cleansing

Now, let's define a function that can help clean the input data. The data refers to some figures like scanned images. We don't have them and so we will remove any such references.

In [108]:
def clean_text(text):
  clean = "\n".join([row for row in text.split("\n")])
  clean = re.sub(r'\(fig[^)]*\)', '', clean, flags=re.IGNORECASE)
  return clean

Let's take this case sheet and extract entities and relations using LLM

## Prompt Definition

**⚠️** You need to duplicate `config.env.example` file in the left and rename as `config.env`. Edit the values in this file and provide the values for API keys and Neo4j credentials

This is a helper function to talk to the LLM with our prompt and text input

In [162]:
def run_text_model(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    prompt: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    return response.text

This is a simple prompt to start with. If the processing is very complex, you can also chain the prompts as and when required. I am going to use a single prompt here that will extract the text strictly as per the Entities and Relationships defined. This is a simplification. 
In the real scenario, especially with medical records, you have to leverage on Domain experts to define the Ontology systematically and capture the important information. You should also be mindful of following the relevant regulations around handling health records,

Instead of one single large model, you can also consider chaining a number of smaller ones as per your needs.

Let's run our completion task with our LLM

In [184]:
import re
from string import Template
from vertexai.preview.language_models import InputOutputTextPair, ChatModel
import json 

def extract_entities_relationships(prompt, tuned_model_name=''):
    try:
        res = run_text_model(project_id, "text-bison@001", 0, 1024, 0.8, 40, prompt, location, tuned_model_name)
        return res
    except Exception as e:
        print(e)
    

In [235]:
prompt="""From the Case sheet for a patient below, extract the following Entities & relationships described in the mentioned format 
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. Do not create new entity types that aren't mentioned below. Document must be summarized and stored inside Case entity under `summary` property. You will have to generate as many entities as needed as per the types below:
    Entity Types:
    label:'Case',id:string //Case
    label:'Person',id:string,age:string,location:string,gender:string //Patient mentioned in the case
    label:'Symptom',id:string,description:string //Symptom Entity; `id` property is the name of the symptom, in lowercase & camel-case & should always start with an alphabet
    label:'Disease',id:string,name:string //Disease diagnosed now or previously as per the Case sheet; `id` property is the name of the disease, in lowercase & camel-case & should always start with an alphabet
    label:'BodySystem',id:string,name:string //Body Part affected. Eg: Chest, Lungs; id property is the name of the part, in lowercase & camel-case & should always start with an alphabet
    label:'Diagnosis',id:string,name:string,description:string,when:string //Diagnostic procedure conducted; `id` property is the summary of the Diagnosis, in lowercase & camel-case & should always start with an alphabet
    label:'Biological',id:string,name:string,description:string //Results identified from Diagnosis; `id` property is the summary of the Biological, in lowercase & camel-case & should always start with an alphabet
2. Next generate each relationships as triples of head, relationship and tail. To refer the head and tail entity, use their respective `id` property. Relationship property should be mentioned within brackets as comma-separated. They should follow these relationship types below. You will have to generate as many relationships as needed as defined below:
    Relationship types:
    case|FOR|person
    person|HAS_SYMPTOM{when:string,frequency:string,span:string}|symptom //the properties inside HAS_SYMPTOM gets populated from the Case sheet
    person|HAS_DISEASE{when:string}|disease //the properties inside HAS_DISEASE gets populated from the Case sheet
    symptom|SEEN_ON|chest
    disease|AFFECTS|heart
    person|HAS_DIAGNOSIS|diagnosis
    diagnosis|SHOWED|biological
3. Summary & description properties inside entities should be the text summary

Output Format (Follow Strictly):
{
    "entities": [{"label":"Case","id":"case1"}, {"label":"Person","id":"person1","age":20,"gender":"female"}],
    "relationships": ["disease|AFFECTS|heart", "case1|FOR|person1", "person|HAS_DISEASE{when:'2020'}|disease"]
}

Question: Now, extract entities & relationships as mentioned above for the text below -
$ctext

Answer:
"""

In [242]:
que = """A 49-year-old woman was admitted to the Department of Radiology of the Second Affiliated Hospital of Zhejiang University in October 2004 with right upper quadrant pain and weight loss.
She was a hepatitis B virus carrier.
Her α-fetoprotein level was 1185.3 ng/mL.
Ultrasonography and computed tomography (CT) revealed a 10-cm mass in the posterior segments of the right liver lobe.
A 1.5-cm mass was also found in the left lateral segment.
These clinical signs indicated that the patient had inoperable HCC and Child-Pugh class A cirrhosis.
TACE was offered to the patient.
Angiogram demonstrated no obvious hepatic arterio-venous shunt, but multiple smaller masses in both lobes of the liver.
An emulsion of oxaliplatin, pirarubicin, hydroxycamptothecin and lipiodol were prepared, 35 mL and 3 mL of the mixture were administered intra-arterially to the right and left hepatic artery, respectively.
The patient experienced right upper quadrant pain after TACE and had an uneventful recovery.
One month later, a second TACE procedure was performed via the right hepatic artery and 40 mL of the mixture was administered.
On the next day, she experienced sudden acute dyspnoea and the peripheral oxygen saturation decreased to 90%.
The chest X-ray showed some increased reticular shadows in the left lung, especially in the lower zones, and a chest CT scan revealed multiple iodized oil-like high-density materials in parenchyma of the lung (Figure ​1).
After 10 mg dexamethasone i.v. and other supportive therapies were administered, the respiratory symptom was attenuated.
Two days later, the patient suffered from a serious headache and transient consciousness loss, accompanying nausea and vomiting followed by confusion, lower extremity weakness.
Non-contrast enhanced CT scanning showed multiple disseminated hyper-intense lesions in the brain, consistent with deposition of iodized oil (Figure ​2).
One week later, her respiratory and neurologic symptoms disappeared completely, and she was discharged.The patient also consequently completed the other three TACE procedures, during which no similar symptoms occurred.

"""
_prompt = Template(prompt).substitute(ctext=clean_text(que))
entity_extraction_tuned_model = 'projects/803648085855/locations/us-central1/models/1971567285113978880'
result = extract_entities_relationships(_prompt, entity_extraction_tuned_model)

result = result.split('Answer:\n ')[1]
result = json.loads(result.replace("\'", "'"))
result

{'entities': [{'label': 'Case', 'id': 'case1'},
  {'label': 'Person', 'id': 'person1', 'age': '49', 'gender': 'female'},
  {'label': 'Disease', 'id': 'disease1', 'name': 'Hepatocellular carcinoma'},
  {'label': 'Symptom', 'id': 'symptom1', 'description': 'upper quadrant pain'},
  {'label': 'Body', 'id': 'body1', 'name': 'chest'},
  {'label': 'Diagnosis',
   'id': 'diagnosis1',
   'name': 'TACE',
   'description': 'transarterial chemoembolization',
   'when': '2014-10-04'},
  {'label': 'Biological',
   'id': 'biological1',
   'name': 'oil-like materials',
   'description': 'iodinated oil-like materials'}],
 'relationships': ['case1|FOR|person1',
  "person1|HAS_SYMPTOM{when:'2014-10-04'}|symptom1",
  'disease1|AFFECTS|body1',
  "disease1|HAS_DISEASE_STAGE{stage:'0'}|disease1",
  'diagnosis1|HAS_BIOLOGICAL|biological1',
  'diagnosis1|HAS_PATIENT|person1']}

## Neo4j Cypher Generation

The entities and relationships we got from the LLM have to be transformed to Cypher so we can write them into Neo4j.

In [243]:
import time

def get_prop_str(prop_dict, _id):
    s = []
    for key, val in prop_dict.items():
      if key != 'label' and key != 'id':
         s.append(_id+"."+key+' = "'+str(val).replace('\"', '"').replace('"', '\"')+'"') 
    return ' ON CREATE SET ' + ','.join(s)

def get_cypher_compliant_var(_id):
    return "_"+ re.sub(r'[\W_]', '', _id)

def generate_cypher(in_json):
    e_map = {}
    e_stmt = []
    r_stmt = []
    e_stmt_tpl = Template("($id:$label{id:'$key'})")
    r_stmt_tpl = Template("""
      MATCH $src
      MATCH $tgt
      MERGE ($src_id)-[:$rel]->($tgt_id)
    """)
    for obj in in_json:
      for j in obj['entities']:
          props = ''
          label = j['label']
          id = j['id']
          if label == 'Case':
                id = 'c'+str(time.time_ns())
          elif label == 'Person':
                id = 'p'+str(time.time_ns())
          varname = get_cypher_compliant_var(j['id'])
          stmt = e_stmt_tpl.substitute(id=varname, label=label, key=id)
          e_map[varname] = stmt
          e_stmt.append('MERGE '+ stmt + get_prop_str(j, varname))

      for st in obj['relationships']:
          rels = st.split("|")
          src_id = get_cypher_compliant_var(rels[0].strip())
          rel = rels[1].strip()
          tgt_id = get_cypher_compliant_var(rels[2].strip())
          stmt = r_stmt_tpl.substitute(
              src_id=src_id, tgt_id=tgt_id, src=e_map[src_id], tgt=e_map[tgt_id], rel=rel)
          
          r_stmt.append(stmt)

    return e_stmt, r_stmt

In [244]:
ent_cyp, rel_cyp = generate_cypher([result])

print(ent_cyp, rel_cyp)

["MERGE (_case1:Case{id:'c1684918150024394055'}) ON CREATE SET ", 'MERGE (_person1:Person{id:\'p1684918150024425570\'}) ON CREATE SET _person1.age = "49",_person1.gender = "female"', 'MERGE (_disease1:Disease{id:\'disease1\'}) ON CREATE SET _disease1.name = "Hepatocellular carcinoma"', 'MERGE (_symptom1:Symptom{id:\'symptom1\'}) ON CREATE SET _symptom1.description = "upper quadrant pain"', 'MERGE (_body1:Body{id:\'body1\'}) ON CREATE SET _body1.name = "chest"', 'MERGE (_diagnosis1:Diagnosis{id:\'diagnosis1\'}) ON CREATE SET _diagnosis1.name = "TACE",_diagnosis1.description = "transarterial chemoembolization",_diagnosis1.when = "2014-10-04"', 'MERGE (_biological1:Biological{id:\'biological1\'}) ON CREATE SET _biological1.name = "oil-like materials",_biological1.description = "iodinated oil-like materials"'] ["\n      MATCH (_case1:Case{id:'c1684918150024394055'})\n      MATCH (_person1:Person{id:'p1684918150024425570'})\n      MERGE (_case1)-[:FOR]->(_person1)\n    ", "\n      MATCH (_p

## Data Ingestion

You will need a Neo4j AuraDS Pro instance.  You can deploy that on Google Cloud Marketplace [here](https://console.cloud.google.com/marketplace/product/endpoints/prod.n4gcp.neo4j.io).

With that complete, you'll need to install the Neo4j library and set up your database connection.

In [ ]:
%pip install graphdatascience

In [ ]:
from graphdatascience import GraphDataScience

In [ ]:
connectionUrl = input('NEO4J_CONN_URL')
username = input('NEO4J_USER')
password = input('NEO4J_PASSWORD')

In [ ]:
gds = GraphDataScience(connectionUrl, auth=(username, password))
gds.version()

Before loading the data, create constraints as below

In [ ]:
gds.run_cypher('CREATE CONSTRAINT unique_case_id IF NOT EXISTS FOR (n:Case) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_person_id IF NOT EXISTS FOR (n:Person) REQUIRE (n.id) IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_symptom_id IF NOT EXISTS FOR (n:Symptom) REQUIRE (n.id) IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_disease_id IF NOT EXISTS FOR (n:Disease) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_bodysys_id IF NOT EXISTS FOR (n:BodySystem) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_diag_id IF NOT EXISTS FOR (n:Diagnosis) REQUIRE n.id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_biological_id IF NOT EXISTS FOR (n:Biological) REQUIRE n.id IS UNIQUE')

Ingest the entities

In [ ]:
%%time
for e in ent_cyp:
    gds.run_cypher(e)


Ingest relationships now

In [ ]:
%%time
for r in rel_cyp:
    gds.run_cypher(r)

This is a helper function to ingest all case sheets inside the `data/` directory

In [ ]:
import glob
def run_pipeline(count=191):
    txt_files = glob.glob("data/case_sheets/*.txt")[0:count]
    print(f"Running pipeline for {len(txt_files)} files")
    failed_files = process_pipeline(txt_files)
    print(failed_files)
    return failed_files

def process_pipeline(files):
    failed_files = []
    for f in files:
        try:
            with open(f, 'r') as file:
                print(f"  {f}: Reading File...")
                data = file.read().rstrip()
                text = clean_text(data)
                print(f"    {f}: Extracting E & R")
                results = extract_entities_relationships(f, text)
                print(f"    {f}: Generating Cypher")
                ent_cyp, rel_cyp = generate_cypher(results)
                print(f"    {f}: Ingesting Entities")
                for e in ent_cyp:
                    gds.run_cypher(e)
                print(f"    {f}: Ingesting Relationships")
                for r in rel_cyp:
                    gds.run_cypher(r)
                print(f"    {f}: Processing DONE")
        except Exception as e:
            print(f"    {f}: Processing Failed with exception {e}")
            failed_files.append(f)
    return failed_files
            
def extract_entities_relationships(f, text):
    start = timer()
    system = "You are a helpful Medical Case Sheet expert who extracts relevant information and store them on a Neo4j Knowledge Graph"
    prompts = [prompt1]
    all_cypher = ""
    results = []
    for p in prompts:
      p = Template(p).substitute(ctext=text)
      res = process_gpt(system, p)
      results.append(json.loads(res))
    end = timer()
    elapsed = (end-start)
    print(f"    {f}: E & R took {elapsed}secs")
    return results

In [ ]:
%%time
failed_files = run_pipeline(200)

If processing failed for some files due to API Rate limit or some other error, you can retry as below

In [ ]:
%%time
failed_files = process_pipeline(failed_files)
failed_files

In [ ]:
results

## Cypher Generation for Consumption

### Tune the model to generate Cypher

In [250]:
training_data = 'gs://gs_vertex_ai/eng2cypher/eng-to-cypher-trng-1.jsonl'
train_steps = 10

vertexai.init(project=project_id, location=location)
model = TextGenerationModel.from_pretrained("text-bison@001")

model.tune_model(
  training_data=training_data,
  train_steps=train_steps,
  tuning_job_location="europe-west4",
  tuned_model_location="us-central1",
)

Creating PipelineJob
PipelineJob created. Resource name: projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524094701
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524094701')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west4/pipelines/runs/tune-large-model-20230524094701?project=803648085855
PipelineJob projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524094701 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524094701 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/803648085855/locations/europe-west4/pipelineJobs/tune-large-model-20230524094701 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/803648085855/locations/europe-west4/pi

### Generate Cypher

In [251]:
def english_to_cypher(prompt, tuned_model_name=''):
    try:
        res = run_text_model(project_id, "text-bison@001", 0, 1024, 0.8, 40, prompt, location, tuned_model_name)
        # res = json.loads(res.replace("\'", "'"))
        return res
    except Exception as e:
        print(e)

In [283]:
prompt = """
Context:
You are an expert Neo4j Cypher translator who understands the question in english and convert it to Cypher based on the Neo4j Schema provided.
Here are the instructions to follow:
1. Use the Neo4j schema to generate cypher compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE keywords in the cypher.
3. Use only Nodes and relationships mentioned in the schema while generating the response
4. Reply ONLY in Cypher when it makes sense.
5. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Heart Disease use `toLower(d.name) contains 'heart disease'`
6. Patient node is synonymous to Person.
Now, use this Neo4j schema and Reply ONLY in Cypher when it makes sense.
Schema:
Nodes:
    label:'Case',id:string,summary:string //Case Node
    label:'Person',id:string,age:string,location:string,gender:string //Patient Node
    label:'Symptom',id:string,description:string //Symptom Node
    label:'Disease',id:string,name:string //Disease Node
    label:'BodySystem',id:string,name:string //Node for Body Part affected Eg: Heart, lungs
    label:'Diagnosis',id:string,name:string,description:string,when:string //Diagnostic Node
    label:'Biological',id:string,name:string,description:string //Node for Results identified from Diagnosis
Relationships:
    (:Case)-[:FOR]->(Person)
    (:Person)-[:HAS_SYMPTOM{when:string,frequency:string,span:string}]->(Symptom)
    (:Person)-[:HAS_DISEASE{when:string}]->(:Disease)
    (:Symptom)-[:SEEN_ON]->(:BodySystem)
    (:Disease)-[:AFFECTS]->(:BodySystem)
    (:Person)-[:HAS_DIAGNOSIS]->(:Diagnosis)
    (:Diagnosis)-[:SHOWED]->(:Biological)

So, for this question: 'How many teenagers have cough related ailments?', you will answer : MATCH (p:Person)-[:HAS_SYMPTOM]->(s:Symptom) WHERE toInteger(p.age) > 12 AND toInteger(p.age) < 20 AND toLower(s.description) CONTAINS 'cough' RETURN COUNT(p) 
Because:
1. As per schema definition of nodes & relationships above, Person node is related to Symptom node via HAS_SYMPTOM relationship.
2. Person's `age` property is a string. So we need to convert to Integer using toInteger before comparing.
3. Teenagers are aged 13 to 19. So we add age filter.
4. Symptoms are stored in `description` property as per schema. And we are doing a case-insensitive match to get the cough symptom
5. Finally, we return the number of patients who match the input criteria using COUNT function


Ouput Format (Strict): //Only code as output. No other text
MATCH (d:Disease) RETURN d.name as disease, SIZE([(d)-[]-(p:Person) | p]) AS affected_patients ORDER BY affected_patients DESC LIMIT 1

Question:
$ctext

Answer:
"""

que = 'How many teenagers have cough related ailments?'
_prompt = Template(prompt).substitute(ctext=clean_text(que))

response = english_to_cypher(_prompt, 'projects/803648085855/locations/us-central1/models/2255294061638320128')
cypher = response.split('Answer:\n ')[1]
    

In [284]:
cypher

"MATCH (p:Person)-[:HAS_SYMPTOM]->(s:Symptom) WHERE toLower(s.description) CONTAINS 'cough' AND p.age > 1 AND p.age < 19 RETURN count(p)"

In [260]:
# To do - move libraries to where they are imported.  Remove unused libraries.

#%pip install graphdatascience
#%pip install python-dotenv
#%pip install retry


# To do - move libraries to where they are imported.  Remove unused libraries.

#import os
#from retry import retry
#import re
#from string import Template
#import json 
#import ast
#import time
#import pandas as pd
#from graphdatascience import GraphDataScience
#import glob
#from timeit import default_timer as timer
#from dotenv import load_dotenv

#from google.cloud import aiplatform
#from google.cloud.aiplatform.private_preview.language_models import ChatModel, InputOutputTextPair
